# 1. Import of input data

In [ ]:
import nbimporter
import sys
import os
from pathlib import Path

In [ ]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 GasEngineHeatPump.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 GasEngineHeatPump.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# Se non trovato, prova nei livelli superiori
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Esegui il file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

In [ ]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Gas Engine Heat Pump')

# 2. Input parameters

In [ ]:
Power = energysystem['Thermal capacity [kW]']
EnergyProduced = energysystem['Energy produced [kWh /year]']
SGUE = energysystem['SGUE']
EnergyInputSource = energysystem['Energy source (Gas-blend, LPG, Biomethane, Hydrogen)']

In [ ]:
LHV = np.where(EnergyInputSource == 'Gas-blend', 36.8,
      np.where(EnergyInputSource == 'LPG', 93,
      np.where(EnergyInputSource == 'Biomethane', 35,
      np.where(EnergyInputSource == 'Hydrogen', 10.8, np.nan))))

HHV = np.where(EnergyInputSource == 'Gas-blend', 40.4,
      np.where(EnergyInputSource == 'LPG', 102,
      np.where(EnergyInputSource == 'Biomethane', 38,
      np.where(EnergyInputSource == 'Hydrogen', 12.7, np.nan))))

In [ ]:
def calculate_mass(Power):
    if Power < 90:
        return (250.490 * Power ** 0.330)
    else:
        i = np.ceil(Power / 90)
        return (250.490 * (Power / i) ** 0.330) * i

mass = Power.apply(calculate_mass)

Lifespan = 20

In [ ]:
i = np.ceil(Power / 90)

conditions = [
    Power < 90,
    Power >= 90
]

choices = [
    0.322 * Power**0.932,
    0.322 * (Power / i)**0.932 * i
]

M_R = np.select(conditions, choices)
massrefrigerant = M_R
mass = mass - massrefrigerant

# 3. Life cycle inventories

## 3.1 Module A1-3

In [ ]:
GEHP_copper = 0.0192709304876261 * mass 
GEHP_aluminium = 0.128643214612436 * mass 
GEHP_lowalloyedsteel = 0.202601386160499 * mass 
GEHP_stainlessSteel = 0.394003982376949 * mass 
GEHP_ferrite = 0.00275299006966087 * mass 
GEHP_brass = 0.227202457102359 * mass 
GEHP_alumina = 0.00254967543377773 * mass 
GEHP_electronics = 0.002 * mass 
GEHP_lubricatingoil = 0.000017 * mass 
GEHP_welding = 0.00012 * mass 
GEHP_waterconsumption = 0.00122 * mass 
GEHP_emissionwaterair = 0.15 * GEHP_waterconsumption
GEHP_emissionwaterwater = 0.02 * mass
GEHP_wastewatertreatment = 0.83 * mass
GEHP_electricityA13 = 0.532 * mass
GEHP_heat = 3.8 * mass
GEHP_hazardouswaste = 0.0006 * mass
GEHP_refrigerantgas = massrefrigerant
GEHP_ethanepentafluoro_methanedifluoro = np.zeros(len(Power))

## 3.2 Module A4

In [ ]:
GEHP_Truk_16_32 = ((500 + 200)*(mass)/1000)
GEHP_Light_Truk = (50*(mass)/1000)

## 3.4 Module B1

In [ ]:
massrefrigerantB2 = np.array(massrefrigerant)
GEHP_maintenancerefrigerantgas = 0.03 * massrefrigerant * Lifespan
GEHP_maintenanceethanepentafluoro_methanedifluoro = 0.03 * massrefrigerant * Lifespan

## 3.5 Module B2

In [ ]:
GEHP_Light_Truk_B2 = np.array(50 * (Lifespan / 2) * 0.01 * (massrefrigerantB2 / 1000))  # Light_Truk_B2 come array

## 3.6 Module B4

In [ ]:
GEHPNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFGEHP = (RSP / ReqSL)  # Scaling factor

## 3.7 Module B6

In [ ]:
GasBlend   = np.where(EnergyInputSource == 'Gas-blend', 1, 0)
Biomethane = np.where(EnergyInputSource == 'Biomethane', 1, 0)
Hydrogen   = np.where(EnergyInputSource == 'Hydrogen', 1, 0)
LPG        = np.where(EnergyInputSource == 'LPG', 1, 0)

In [ ]:
EnergyProduced = np.array(EnergyProduced)
EnergyProducedforGas = EnergyProduced

y = (EnergyProduced * 3.6 * RSP).flatten()

# REPARTITION IN THE ENERGY SOURCE
# Costants
k = 0.0256410256410256

# Base terms calculations
base = (1 / SGUE) / (HHV)
h2_term = (1 / (SGUE * (12.7 / 0.09)))

conditions = [
    (EnergyInputSource == 'Gas-blend') & (Year == 2025),
    (EnergyInputSource == 'Gas-blend') & (Year == 2030),
    (EnergyInputSource == 'Gas-blend') & (Year == 2035),
    (EnergyInputSource == 'Gas-blend') & (Year == 2040),
    (EnergyInputSource == 'Gas-blend') & (Year == 2045),
    (EnergyInputSource == 'Gas-blend') & (Year == 2050),
    (EnergyInputSource == 'LPG'),
    (EnergyInputSource == 'Biomethane'),
    (EnergyInputSource == 'Hydrogen')
]

values = [
    base,
    base * 0.90 + base * 0.084 + h2_term * 0.014,
    base * 0.795 + base * 0.189 + h2_term * 0.016,
    base * 0.688 + base * 0.294 + h2_term * 0.018,
    base * 0.582 + base * 0.399 + h2_term * 0.020,
    base * 0 + base * 0.25 + h2_term * 0.20,
    (1 / SGUE) / (HHV),
    (1 /SGUE) / (HHV),
    (1 / (SGUE * (12.7 / 0.09)))
]

x = np.select(conditions, values, default=np.nan)

GEHP_EnergyVectorNG = (1 / (SGUE * HHV)) * EnergyProduced * RSP * A * GasBlend
GEHP_EnergyVectorBiomethane = ((1 / (SGUE * HHV)) * EnergyProduced * RSP) * B * GasBlend
GEHP_EnergyVectorHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * EnergyProduced * RSP) * C * GasBlend #120 MJ/kg
GEHP_EnergyVectorBiomethaneInput = ((1 / (SGUE * HHV)) * EnergyProduced * RSP) * Biomethane
GEHP_EnergyVectorLPGinput = ((1 / (SGUE * HHV)) * EnergyProduced * RSP) * LPG
GEHP_EnergyVectorHydrogeninput = ((((1 / 0.9) * (1 / 3.6)) * 120) * EnergyProduced * RSP) * Hydrogen 

GEHP_electricityB6 = 0.0404 * (1 / SGUE) * EnergyProduced * RSP
GEHP_acetaldehyde = 0.000000001 * (1 / SGUE) * EnergyProduced * RSP
GEHP_aceticacid = 0.00000015 * (1 / SGUE)  * EnergyProduced * RSP
GEHP_benzene = 0.0000004 * (1 / SGUE) * EnergyProduced * RSP
GEHP_benzoapyrene = 0.00000000001 * (1 / SGUE) * EnergyProduced * RSP
GEHP_butane = 0.0000007 * (1 / SGUE) * EnergyProduced * RSP
GEHP_carbonDioxideFossil = 0.05406 * (1 / SGUE) * EnergyProduced * RSP
GEHP_carbonMonoxideFossil = 0.00003 * (1 / SGUE) * EnergyProduced * RSP
GEHP_dinitrogenMonoxide = 0.0000005 * (1 / SGUE) * EnergyProduced * RSP
GEHP_formaldehyde = 0.0000001 * (1 / SGUE) * EnergyProduced * RSP
GEHP_mercury = 0.00000000003 * (1 / SGUE) * EnergyProduced * RSP
GEHP_methaneFossil = 0.000002 * (1 / SGUE) * EnergyProduced * RSP
GEHP_nitrogenOxides = 0.00002 * (1 / SGUE) * EnergyProduced * RSP
GEHP_PAH = 0.00000001 * (1 / SGUE) * EnergyProduced * RSP
GEHP_particulates = 0.0000001 * (1 / SGUE) * EnergyProduced * RSP
GEHP_pentane = 0.0000012 * (1 / SGUE) * EnergyProduced * RSP
GEHP_propane = 0.0000002 * (1 / SGUE) * EnergyProduced * RSP
GEHP_propionicAcid = 0.00000002 * (1 / SGUE) * EnergyProduced * RSP
GEHP_sulfurDioxide = 0.0000005 * (1 / SGUE) * EnergyProduced * RSP
GEHP_toluene = 0.0000002 * (1 / SGUE) * EnergyProduced * RSP
GEHP_nitrate = 0.00000013 * (1 / SGUE) * EnergyProduced * RSP
GEHP_nitrite = 0.000000003 * (1 / SGUE) * EnergyProduced * RSP
GEHP_sulfate = 0.00000005 * (1 / SGUE) * EnergyProduced * RSP
GEHP_sulfite = 0.00000005 * (1 / SGUE) * EnergyProduced * RSP
GEHP_usephaserefrigerantgas = 0.04 * massrefrigerant * RSP
GEHP_usephaseethanepentafluoro_methanedifluoro = 0.04 * massrefrigerant * RSP

## 3.8 Module C2

In [ ]:
GEHP_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.9 Module C3

In [ ]:
GEHP_EOL_copper = GEHP_copper
GEHP_EOL_aluminium = GEHP_aluminium
GEHP_EOL_lowalloyedsteel = GEHP_lowalloyedsteel
GEHP_EOL_stainlessSteel = GEHP_stainlessSteel
GEHP_EOL_ferrite = GEHP_ferrite 
GEHP_EOL_brass = GEHP_brass 
GEHP_EOL_alumina = GEHP_alumina
GEHP_EOL_electronics = GEHP_electronics
GEHP_EOL_lubricatingoil = GEHP_lubricatingoil
GEHP_EOL_hazardouswaste = GEHP_hazardouswaste
GEHP_EOL_refrigerantgas = massrefrigerant

## 3.10 Module C4

In [ ]:
GEHP_EOL_landfill_copper = GEHP_EOL_copper * LandfillMetals
GEHP_EOL_landfill_aluminium = GEHP_EOL_aluminium * LandfillMetals
GEHP_EOL_landfill_lowalloyedsteel = GEHP_EOL_lowalloyedsteel * LandfillMetals
GEHP_EOL_landfill_stainlessSteel = GEHP_EOL_stainlessSteel * LandfillMetals2
GEHP_EOL_landfill_ferrite  = GEHP_EOL_ferrite  * LandfillMetals
GEHP_EOL_landfill_brass  = GEHP_EOL_brass  * LandfillMetals
GEHP_EOL_landfill_alumina  = GEHP_EOL_alumina  * LandfillMetals
GEHP_EOL_inc_electronics = GEHP_EOL_electronics * IncinerationElectronics #kg
GEHP_EOL_landfill_electronics = GEHP_EOL_electronics * LandfillElectronics #kg
GEHP_EOL_inc_lubricatingoil = GEHP_EOL_lubricatingoil * IncinerationLubricatingOil #kg
GEHP_EOL_inc_hazardouswaste = GEHP_hazardouswaste * IncinerationHazardous #kg
GEHP_EOL_landfill_refrigerantgas = GEHP_EOL_refrigerantgas * LandfillRefrigerant

## 3.11 Module D1 (related to the export of secondary materials)

In [ ]:
GEHP_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * GEHP_EOL_copper) - (RecycledContentMetals * GEHP_EOL_copper)
GEHP_EOL_primary_copper = ((RecycledContentMetals * GEHP_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * GEHP_EOL_copper)) * SRmetals

GEHP_EOL_recycling_aluminium = (RecyclingMetals * RecyclingEfficiencyMetals * GEHP_EOL_aluminium) - (RecycledContentMetals * GEHP_EOL_aluminium)
GEHP_EOL_primary_aluminium = ((RecycledContentMetals * GEHP_EOL_aluminium) - (RecyclingMetals * RecyclingEfficiencyMetals * GEHP_EOL_aluminium)) * SRmetals

GEHP_EOL_recycling_lowalloyedsteel = (RecyclingMetals * RecyclingEfficiencyMetals * GEHP_EOL_lowalloyedsteel) - (RecycledContentMetals * GEHP_EOL_lowalloyedsteel)
GEHP_EOL_primary_lowalloyedsteel = ((RecycledContentMetals * GEHP_EOL_lowalloyedsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * GEHP_EOL_lowalloyedsteel)) * SRmetals

GEHP_EOL_recycling_stainlessSteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * GEHP_EOL_stainlessSteel) - (RecycledContentMetals * GEHP_EOL_stainlessSteel)
GEHP_EOL_primary_stainlessSteel = ((RecycledContentMetals * GEHP_EOL_stainlessSteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * GEHP_EOL_stainlessSteel)) * SRmetals


## 3.12 Module D3 (related to the export of energy as result of waste incineration)

In [ ]:
GEHP_EOL_energyrecoveryheat_electronics = - GEHP_EOL_inc_electronics * LHVElectronics * Xincheat
GEHP_EOL_energyrecoveryele_electronics = - GEHP_EOL_inc_electronics * LHVElectronics * Xincele

GEHP_EOL_energyrecoveryheat_lubricatingoil = - GEHP_EOL_inc_lubricatingoil * LHVHazardous * Xincheat
GEHP_EOL_energyrecoveryele_lubricatingoil = - GEHP_EOL_inc_lubricatingoil * LHVHazardous * Xincele

GEHP_EOL_energyrecoveryheat_hazardouswaste = - GEHP_EOL_inc_hazardouswaste * LHVHazardous * Xincheat
GEHP_EOL_energyrecoveryele_hazardouswaste = - GEHP_EOL_inc_hazardouswaste * LHVHazardous * Xincele

# 4. Data for export

In [ ]:
ThermalEnergyProduced_GEHP = np.array([EnergyProduced * RSP])